In [ ]:
!pip install cantera
import matplotlib.pyplot as plt
import cantera as ct
import numpy as np

def integrate(grid, arr):
    """Gets integral of array over a given grid with trapezoidal.
    """
    integral = 0.0
    if len(grid) == 1:
        return integral

    #
    # Trapezoidal rule
    #
    d_grid = np.diff(grid)
    mid_value = (np.array(arr[:-1]) + np.array(arr[1:])) / 2

    return np.sum(d_grid * mid_value)


In [ ]:
# Define the gas mixture
fuel = {"H2": 1.0}  # X_{fuel}
oxidizer = {"O2": 1.0, "N2": 3.76} # X_{oxidizer}
T = 300.0
p = ct.one_atm
equivalence_ratio = 1

In [ ]:
#
# Run simulation over a range of flame widths
#

cases = {}
widths = [0.1, 0.3, 0.6, 1, 2.0, 4.0, 6.0] # m
for width in widths:
    cases[str(width)] = {}
    cases[str(width)]["mechanism"] = "gri30.yaml"
    cases[str(width)]["transport_model"] = "mixture-averaged"
    cases[str(width)]["width"] = width

for case in cases:
    
    gas = ct.Solution(cases[case]["mechanism"])
    gas.TPX = T, p, fuel
    gas.set_equivalence_ratio(equivalence_ratio, fuel, oxidizer)
    flame = ct.FreeFlame(gas, width=cases[case]["width"])
    flame.transport_model = cases[case]["transport_model"]

    flame.energy_enabled = True
    flame.set_refine_criteria(ratio=2, slope=0.05, curve=0.05, prune=0.01)

    flame.set_max_jac_age(10, 10)
    flame.set_time_step(1e-8, [10, 20, 40, 80, 100, 100, 150])
    flame.max_time_step_count = 5000    

    # Set steady-state tolerance to relative and absolute tolerances of 1e-15
    flame.flame.set_steady_tolerances(default=[1.0e-5, 1.0e-9])
    flame.flame.set_transient_tolerances(default=[1.0e-5, 1.0e-9])

    flame.solve(loglevel=0, auto=True)

    cases[case]["flame"] = flame

In [ ]:
#
# Plot equilibrium solutions
#
fig, axs = plt.subplots(2, 1, figsize=(4, 4))

ax = axs[0]
for case in cases:
    ax.plot(
        cases[case]["flame"].grid,
        cases[case]["flame"].T,
        label=f"$W$={cases[case]['flame'].grid[-1]:.1f} $[m]$",
    )
ax.set_ylabel(r"$T$ $[K]$")
ax.grid(True)

ax = axs[1]
for case in cases:
    ax.plot(
        cases[case]["flame"].grid, 
        cases[case]["flame"].velocity, 
    )
ax.set_ylabel(r"$u$ $[m/s]$")
ax.set_xlabel(r"$x$ $[m]$")
ax.grid(True)
# Put legend outside of plot
fig.legend(bbox_to_anchor=(0.9, 0.5), loc="center left")

In [ ]:
#
# Plot equilibrium solutions on scaled domain
#
fig, axs = plt.subplots(2, 1, figsize=(4, 4))

ax = axs[0]
for case in cases:
    ax.plot(
        cases[case]["flame"].grid / cases[case]["flame"].grid[-1],
        cases[case]["flame"].T,
        label=f"$W$={cases[case]['width']:.1f} m",
    )
ax.set_ylabel(r"$T$ $[K]$")
ax.grid(True)

ax = axs[1]
for case in cases:
    ax.plot(
        cases[case]["flame"].grid / cases[case]["flame"].grid[-1],
        cases[case]["flame"].velocity,
        )
ax.set_ylabel(r"$u$ $[m/s]$")
ax.set_xlabel(r"$x / W$ $[-]$")
ax.grid(True)
# Put legend outside of plot
fig.legend(bbox_to_anchor=(0.9, 0.5), loc="center left")

In [ ]:
#
# Plot NOx emissions
#
fig, axs = plt.subplots(2, 1, figsize=(4, 4))

ax = axs[0]
for case in cases:
    ax.plot(
        cases[case]["flame"].grid / cases[case]["flame"].grid[-1],
        cases[case]["flame"].Y[gas.species_index("NO"), :],
        label=f"$W$={cases[case]['width']:.1f} m",
    )
ax.set_ylabel(r"$Y_{NO}$ $[-]$")
ax.grid(True)

ax = axs[1]
for case in cases:
    ax.plot(
        cases[case]["flame"].grid / cases[case]["flame"].grid[-1],
        cases[case]["flame"].net_production_rates[gas.species_index("NO"), :],
        )

ax.set_ylabel(r"$\dot{\omega_{NO}}$ $[]$")
ax.set_xlabel(r"$x / W$ $[-]$")
ax.grid(True)
# Put legend outside of plot
fig.legend(bbox_to_anchor=(0.9, 0.5), loc="center left")

In [ ]:
#
# Calculate residence time for each case
#
residence_times = []
for case in cases:

    flame = cases[case]["flame"]
    Y_NO = flame.Y[gas.species_index("NO"), :]
    grid = flame.grid

    # Lagrangian residence time dt = dx / u
    residence_times.append(np.zeros_like(grid))

    u_minus_1 = 1.0 / flame.velocity[: len(Y_NO)]
    for i_x, x in enumerate(grid):
        residence_times[-1][i_x] = integrate(
            grid[: i_x], u_minus_1[: i_x]
        )

# Plot equilibrium solutions
fig, axs = plt.subplots(1, 1, figsize=(4, 4))

case = cases[list(cases.keys())[-1]]
residence_time = residence_times[-1]

#for case, residence_time in zip(cases, residence_times):

# Center at NO > 0
flame = case["flame"]
residence_time_c = residence_time - residence_time[np.argmax(Y_NO > 0.0001)]

axs.plot(
    residence_time_c,
    flame.Y[gas.species_index("NO"), :],
)

# Plot T vs. residence time
ax_t = axs.twinx()
ax_t.plot(
    residence_time_c,
    flame.T,
    color="C1",
)

axs.set_ylabel(r"$Y_{NO}$ $[-]$", color="C0")
ax_t.set_ylabel(r"$T$ $[K]$", color="C1")
axs.grid(True)
axs.set_xlabel(r"$\tau$ $[s]$")
axs.set_xlim([-0.2, 0.2])